# SVM para test luminaria


In [79]:
import pandas as pd
import numpy as np

from typing import List
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


In [80]:
df=pd.read_csv('datasets/luminaria_con_niveles_de_riesgo 231.csv')
df.head()

,semestre_ord,desmotivacion_bin,considerado_abandonar_bin,dificultades_economicas_bin,empleo_ord,impacto_laboral_ord,reprobo_materias_bin,apoyo_institucional_ord,satisfaccion_servicios_ord,actividades_extracurriculares_ord,nivel_riesgo
0,7mo o más,No,Sí,No,"Sí, medio tiempo",Algo,No,Algunas veces,Insatisfecho/a,Algunas veces,Alto Riesgo
1,7mo o más,No,Sí,Sí,"Sí, medio tiempo",Algo,No,Siempre,Insatisfecho/a,"Sí, frecuentemente",Alto Riesgo
2,1er - 3er,Si,Sí,Sí,No,Algo,No,Algunas veces,Insatisfecho/a,Nunca,Medio Riesgo
3,1er - 3er,Si,No,Sí,"Sí, medio tiempo",Algo,No,Algunas veces,Satisfecho/a,Algunas veces,Alto Riesgo
4,7mo o más,No,No,No,"Sí, medio tiempo",No afecta/No trabajo,No,Algunas veces,Satisfecho/a,"Sí, frecuentemente",Bajo Riesgo


In [81]:
# dividimos el data set en 3 partes
# entrenamiento 70%, validacion 15% y prueba 15%

from sklearn.model_selection import train_test_split
df_train, df_temp = train_test_split(df, test_size=0.3, random_state=42, stratify=df['nivel_riesgo'],)
df_val, df_test = train_test_split(df_temp, test_size=0.5, random_state=42, stratify=df_temp['nivel_riesgo'],)
print(f'Tamaño del conjunto de entrenamiento: {len(df_train)}')
print(f'Tamaño del conjunto de validación: {len(df_val)}')
print(f'Tamaño del conjunto de prueba: {len(df_test)}')

Tamaño del conjunto de entrenamiento: 161
Tamaño del conjunto de validación: 35
Tamaño del conjunto de prueba: 35


In [82]:
X_train = df_train.drop('nivel_riesgo', axis=1)
y_train = df_train['nivel_riesgo']

X_val = df_val.drop('nivel_riesgo', axis=1)
y_val = df_val['nivel_riesgo']

X_test = df_test.drop('nivel_riesgo', axis=1)
y_test = df_test['nivel_riesgo']

In [88]:
X_train.head()

,semestre_ord,desmotivacion_bin,considerado_abandonar_bin,dificultades_economicas_bin,empleo_ord,impacto_laboral_ord,reprobo_materias_bin,apoyo_institucional_ord,satisfaccion_servicios_ord,actividades_extracurriculares_ord
217,1er - 3er,Si,Sí,No,No,No afecta/No trabajo,Sí,Algunas veces,Insatisfecho/a,Nunca
104,7mo o más,Si,Sí,Sí,"Sí, medio tiempo",Algo,Sí,Algunas veces,Satisfecho/a,Algunas veces
19,7mo o más,No,Sí,No,"Sí, medio tiempo","Sí, mucho",No,Siempre,Muy satisfecho/a,Nunca
129,7mo o más,No,Sí,No,No,No afecta/No trabajo,No,Nunca,Insatisfecho/a,Nunca
84,7mo o más,No,No,No,No,No afecta/No trabajo,No,Siempre,Satisfecho/a,Algunas veces


In [89]:
X_train.columns

Index(['semestre_ord', 'desmotivacion_bin', 'considerado_abandonar_bin',
       'dificultades_economicas_bin', 'empleo_ord', 'impacto_laboral_ord',
       'reprobo_materias_bin', 'apoyo_institucional_ord',
       'satisfaccion_servicios_ord', 'actividades_extracurriculares_ord'],
      dtype='object')

In [98]:
X_train['actividades_extracurriculares_ord'].value_counts()

actividades_extracurriculares_ord
Algunas veces         65
Nunca                 62
Sí, frecuentemente    34
Name: count, dtype: int64

In [109]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder

# 1. Definir las columnas según encoding_svm_bayes.json
binarias = ['dificultades_economicas_bin', 'reprobo_materias_bin']
ordinales = {
    'semestre_ord': ['1er - 3er', '4to - 6to', '7mo o más'],
    'apoyo_institucional_ord': ['Nunca', 'Algunas veces', 'Siempre'],
    'satisfaccion_servicios_ord': ['Muy insatisfecho/a', 'Insatisfecho/a', 'Satisfecho/a', 'Muy satisfecho/a'],
    'actividades_extracurriculares_ord': ['Nunca', 'Algunas veces', 'Sí, frecuentemente']
}
onehot = ['empleo_ord', 'impacto_laboral_ord']

# 2. Transformador para variables binarias
class BinarioMapper(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.maps = {
            'dificultades_economicas_bin': {'Sí': 1, 'No': 0},
            'reprobo_materias_bin': {'Sí': 1, 'No': 0}
        }
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X_ = X.copy()
        for col, mapping in self.maps.items():
            X_[col + '_bin'] = X_[col].map(mapping)
        return X_[[col + '_bin' for col in self.maps]]

# 3. Crear el ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        # Binarias
        ('bin', BinarioMapper(), binarias),
        # Ordinales
        ('ord', OrdinalEncoder(categories=[ordinales[k] for k in ordinales], dtype=int),
         list(ordinales.keys())),
        # One-hot
        ('onehot', OneHotEncoder(drop='first', sparse_output=False, dtype=int), onehot)
    ]
)

pipeline = Pipeline([
    ('pre', preprocessor),
    ('scaler', StandardScaler())
])



In [115]:

# prubea
X_train_prep = pipeline.fit_transform(X_train)
print(f'Shape después del pipeline: {X_train_prep.shape}')


Shape después del pipeline: (161, 10)


In [ ]:
# extraer nombres de columnas después del preprocesamiento
bin_cols = [f"{col}_bin" for col in binarias]
ord_cols = list(ordinales.keys())
# este es el mas importante, pues los nombres dependen de los datos
onehot_cols = pipeline.named_steps['pre'].named_transformers_['onehot'].get_feature_names_out(onehot)
all_cols = bin_cols + ord_cols + list(onehot_cols)
all_cols

['dificultades_economicas_bin_bin',
 'reprobo_materias_bin_bin',
 'semestre_ord',
 'apoyo_institucional_ord',
 'satisfaccion_servicios_ord',
 'actividades_extracurriculares_ord',
 'empleo_ord_Sí, medio tiempo',
 'empleo_ord_Sí, tiempo completo',
 'impacto_laboral_ord_No afecta/No trabajo',
 'impacto_laboral_ord_Sí, mucho']

In [113]:
X_train_encoded = pd.DataFrame(
    pipeline.transform(X_train),
    columns=all_cols,
    index=X_train.index
)

X_val_encoded = pd.DataFrame(
    pipeline.transform(X_val),
    columns=all_cols,
    index=X_val.index
)

X_test_encoded = pd.DataFrame(
    pipeline.transform(X_test),
    columns=all_cols,
    index=X_test.index
)

print(f'Cojunto de entrenamiento preprocesado: {X_train_encoded.shape}')
print(f'Cojunto de validación preprocesado: {X_val_encoded.shape}')
print(f'Cojunto de pruebas preprocesado: {X_test_encoded.shape}')

Cojunto de entrenamiento preprocesado: (161, 10)
Cojunto de validación preprocesado: (35, 10)
Cojunto de pruebas preprocesado: (35, 10)


In [112]:
pd.DataFrame(X_train_prep, columns=all_cols, index=X_train.index).head()

,dificultades_economicas_bin_bin,reprobo_materias_bin_bin,semestre_ord,apoyo_institucional_ord,satisfaccion_servicios_ord,actividades_extracurriculares_ord,"empleo_ord_Sí, medio tiempo","empleo_ord_Sí, tiempo completo",impacto_laboral_ord_No afecta/No trabajo,"impacto_laboral_ord_Sí, mucho"
217,-0.613244,1.799123,-1.371719,-0.024329,-0.967849,-1.098012,-0.680908,-0.270801,0.791368,-0.283790
104,1.630672,1.799123,0.905051,-0.024329,0.574960,0.231160,1.468626,-0.270801,-1.263635,-0.283790
19,-0.613244,-0.555826,0.905051,1.934141,2.117768,-1.098012,1.468626,-0.270801,-1.263635,3.523729
129,-0.613244,-0.555826,0.905051,-1.982798,-0.967849,-1.098012,-0.680908,-0.270801,0.791368,-0.283790
84,-0.613244,-0.555826,0.905051,1.934141,0.574960,0.231160,-0.680908,-0.270801,0.791368,-0.283790


## SVM


In [144]:
from sklearn.svm import SVC

## SVM
svm_clf = SVC(kernel='rbf', C=1 , gamma=0.01)

svm_clf.fit(X_train_encoded, y_train)



,C,1
,kernel,'rbf'
,degree,3
,gamma,0.01
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


In [145]:
y_pred = svm_clf.predict(X_val_encoded)
y_pred

array(['Alto Riesgo', 'Bajo Riesgo', 'Alto Riesgo', 'Medio Riesgo',
       'Alto Riesgo', 'Alto Riesgo', 'Medio Riesgo', 'Bajo Riesgo',
       'Bajo Riesgo', 'Bajo Riesgo', 'Alto Riesgo', 'Bajo Riesgo',
       'Alto Riesgo', 'Bajo Riesgo', 'Bajo Riesgo', 'Bajo Riesgo',
       'Alto Riesgo', 'Bajo Riesgo', 'Medio Riesgo', 'Medio Riesgo',
       'Bajo Riesgo', 'Alto Riesgo', 'Bajo Riesgo', 'Medio Riesgo',
       'Alto Riesgo', 'Medio Riesgo', 'Alto Riesgo', 'Bajo Riesgo',
       'Bajo Riesgo', 'Bajo Riesgo', 'Alto Riesgo', 'Medio Riesgo',
       'Bajo Riesgo', 'Bajo Riesgo', 'Bajo Riesgo'], dtype=object)

In [146]:
from sklearn.metrics import f1_score
f1 = f1_score(y_val, y_pred, average='weighted')
print(f'F1-score en el conjunto de validación: {f1:.4f}')

F1-score en el conjunto de validación: 0.8509
